# Panel Regression

A panel regression is a suitable regression method for analyzing our data, especially since we have quarterly data for multiple companies over a long period of time. A panel regression model accounts for both within-entity and between-entity variations, making it a useful tool to analyze data with a time series and cross-sectional dimension.  
With panel data, we can control for individual-level characteristics that may affect forecast accuracy by including fixed effects for each company or industry. We can also account for time-specific factors that may affect forecast accuracy by including time fixed effects or time-varying covariates.

### Random Effects Panel Regression

In [87]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [88]:
df = pd.read_csv("Dataframes/economic_story_regression.csv")
df["Date"] = pd.to_datetime(df["Date"])
df

,Instrument,Date,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Number of Analysts,Recommendation - Mean (1-5),...,Price Target - Standard Deviation,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status,CBOE Crude Oil ETF Volatility Index,90-Day AA Financial Commercial Paper Interest Rate,Inflation Risk Premium,"University of Michigan: Consumer Sentiment, Index 1966:Q1=100",Unemployment Rate
0,AVY.N,2013-01-01,11.178,11.178,Materials,3.625620,3.426001e+09,1.532200e+09,7.0,3.00000,...,2.44949,0.799347,10.637479,207295.0,-1,22.538500,0.166000,0.359537,76.666667,7.733333
1,AVY.N,2013-04-01,2.482,2.482,Materials,2.046169,4.309428e+09,1.498900e+09,7.0,2.87500,...,3.92272,0.895787,24.148726,0.0,-1,23.468125,0.143750,0.360572,81.666667,7.533333
2,AVY.N,2013-07-01,1.068,1.068,Materials,1.551601,4.258229e+09,1.552300e+09,8.0,2.87500,...,4.30946,0.906078,-0.054218,0.0,-1,24.099219,0.125625,0.463789,81.566667,7.233333
3,AVY.N,2013-10-01,8.095,8.095,Materials,2.775994,4.273414e+09,1.504900e+09,7.0,2.62500,...,3.45230,0.903176,2.467620,236291.0,-1,19.740938,0.125806,0.441854,76.933333,6.933333
4,AVY.N,2014-01-01,1.471,1.471,Materials,3.204412,4.877168e+09,1.583900e+09,8.0,2.33333,...,3.42555,0.920659,16.003852,157761.0,-1,19.563770,0.126230,0.465460,80.933333,6.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,17.194,17.194,Retailing,2.836295,1.742020e+10,1.411448e+09,8.0,2.30000,...,50.48823,3.684400,-5.130322,157740.0,-1,43.036308,0.140755,0.382645,69.900000,4.200000
19197,POOL.OQ,2022-01-01,40.267,40.267,Retailing,3.269867,2.268979e+10,1.035557e+09,8.0,2.30000,...,41.33833,4.504382,30.489705,82349.0,-1,52.695806,0.467234,0.380946,63.133333,3.800000
19198,POOL.OQ,2022-04-01,34.342,34.342,Retailing,6.630736,1.696712e+10,1.412650e+09,9.0,2.00000,...,37.49206,3.422880,-25.160664,290897.0,-1,50.457581,1.394118,0.495588,57.866667,3.600000
19199,POOL.OQ,2022-07-01,1.503,1.503,Retailing,3.625249,1.405795e+10,2.055818e+09,9.0,2.20000,...,57.57373,2.749481,-21.661570,185584.0,-1,49.861562,2.885882,0.423045,56.100000,3.566667


Standardising

Standardization assumes that your data has a Gaussian (bell curve) distribution. This does not strictly have to be true, but the technique is more effective if your attribute distribution is Gaussian. Standardization is useful when your data has varying scales and the algorithm you are using does make assumptions about your data having a Gaussian distribution, such as linear regression, logistic regression, and linear discriminant analysis.

In [89]:
rescale =  df.copy()
rescale["Revenue - Actual"] = StandardScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Market Capitalization"] = StandardScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = StandardScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = StandardScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = StandardScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))
rescale["Earnings Per Share – Coefficient of Variation"] = StandardScaler().fit_transform(np.array(rescale["Earnings Per Share – Coefficient of Variation"]).reshape(-1,1))
rescale["Number of Analysts"] = StandardScaler().fit_transform(np.array(rescale["Number of Analysts"]).reshape(-1,1))
rescale["Recommendation - Mean (1-5)"] = StandardScaler().fit_transform(np.array(rescale["Recommendation - Mean (1-5)"]).reshape(-1,1))
rescale["Recommendation change"] = StandardScaler().fit_transform(np.array(rescale["Recommendation change"]).reshape(-1,1))
rescale["Recommendation - Mean (1-5).1"] = StandardScaler().fit_transform(np.array(rescale["Recommendation - Mean (1-5).1"]).reshape(-1,1))
rescale["Price Target - Standard Deviation"] = StandardScaler().fit_transform(np.array(rescale["Price Target - Standard Deviation"]).reshape(-1,1))
rescale["CBOE Crude Oil ETF Volatility Index"] = StandardScaler().fit_transform(np.array(rescale["CBOE Crude Oil ETF Volatility Index"]).reshape(-1,1))
rescale["90-Day AA Financial Commercial Paper Interest Rate"] = StandardScaler().fit_transform(np.array(rescale["90-Day AA Financial Commercial Paper Interest Rate"]).reshape(-1,1))
rescale["Inflation Risk Premium"] = StandardScaler().fit_transform(np.array(rescale["Inflation Risk Premium"]).reshape(-1,1))
rescale["University of Michigan: Consumer Sentiment, Index 1966:Q1=100"] = StandardScaler().fit_transform(np.array(rescale["University of Michigan: Consumer Sentiment, Index 1966:Q1=100"]).reshape(-1,1))
rescale["Unemployment Rate"] = StandardScaler().fit_transform(np.array(rescale["Unemployment Rate"]).reshape(-1,1))
#rescale

log transformation with a log shift of 1

In [90]:
rescale["Earnings Per Share - Actual Surprise AbsVals"] = np.log(rescale["Earnings Per Share - Actual Surprise AbsVals"] + 1)
#rescale

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [91]:
#reformatting indices of dataframe for panel regression
rescale['Instrument'] = rescale['Instrument'].astype('category')
rescale['Date'] = pd.to_datetime(rescale['Date'])

# set the index to be the time variable and the cross-sectional variable
rescale.set_index(['Instrument', 'Date'], inplace=True)

In [92]:
# drop rows with NaNs or Infs
#rescale.dropna(inplace=True, how='all')

# separate the dependent variable and independent variables
y = rescale.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = rescale.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


/Users/ginevrabramante/opt/anaconda3/envs/AnalystForecast/lib/python3.11/site-packages/linearmodels/panel/model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_96893/4041962935.py:12: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

CBOE Crude Oil ETF Volatility Index, 90-Day AA Financial Commercial Paper Interest Rate, Inflation Risk Premium, University of Michigan: Consumer Sentiment, Index 1966:Q1=100, Unemployment Rate

  fixed_effects_results = fixed_effects_model.fit()


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0185
Estimator:,PanelOLS,R-squared (Between):,0.0188
No. Observations:,18117,R-squared (Within):,0.0211
Date:,"Tue, Mar 07 2023",R-squared (Overall):,0.0163
Time:,20:16:09,Log-likelihood,-2.36e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,30.085
Entities:,502,P-value,0.0000
Avg Obs:,36.090,Distribution:,"F(11,17566)"
Min Obs:,0.0000,,
Max Obs:,40.000,F-statistic (robust):,8868.6


### Characteristics

In [93]:
df_c = pd.read_csv("Dataframes/characteristics_regression.csv")
df_c["Date"] = pd.to_datetime(df_c["Date"])
df_c

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,11.178,1.532200e+09,3.426001e+09,0.799347,10.637479,207295.0,-1
1,AVY.N,2013-04-01,Materials,2.482,2.482,1.498900e+09,4.309428e+09,0.895787,24.148726,0.0,-1
2,AVY.N,2013-07-01,Materials,1.068,1.068,1.552300e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
3,AVY.N,2013-10-01,Materials,8.095,8.095,1.504900e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
4,AVY.N,2014-01-01,Materials,1.471,1.471,1.583900e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
...,...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-10-01,Retailing,17.194,17.194,1.411448e+09,1.742020e+10,3.684400,-5.130322,157740.0,-1
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,1.035557e+09,2.268979e+10,4.504382,30.489705,82349.0,-1
19198,POOL.OQ,2022-04-01,Retailing,34.342,34.342,1.412650e+09,1.696712e+10,3.422880,-25.160664,290897.0,-1
19199,POOL.OQ,2022-07-01,Retailing,1.503,1.503,2.055818e+09,1.405795e+10,2.749481,-21.661570,185584.0,-1


# With Covid

In [94]:
rescale = rescale.dropna()
rescale.set_index(['Instrument', 'Date'], inplace=True)
rescale

KeyError: "None of ['Instrument', 'Date'] are in the columns"

In [ ]:
y = rescale["Earnings Per Share - Actual Surprise AbsVals"]
X = rescale.loc[:, "Earnings Per Share – Coefficient of Variation":]
# perform the fixed effects panel regression, setting check_rank=False
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True, check_rank=False)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


# Without covid

In [ ]:
df = pd.read_csv("Dataframes/economic_story_regression.csv")
df["Date"] = pd.to_datetime(df["Date"])
n = 4
rescale = df.iloc[:, n:].copy()

scaler = MinMaxScaler()

for col in rescale.columns:
    if rescale[col].dtype != "object":
        rescale[col] = scaler.fit_transform(rescale[col].values.reshape(-1, 1))

rescale = pd.concat([df.iloc[:,:n], rescale], axis=1)
rescale = rescale.dropna()
rescale

In [ ]:
covid_start = pd.to_datetime("2020-01-01")

df_covid = rescale[rescale["Date"] >= covid_start]

df_nocovid = rescale[rescale["Date"] < covid_start]
df_nocovid

In [ ]:
df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])

# set the index to be the time variable and the cross-sectional variable
df_nocovid.set_index(['Instrument', 'Date'], inplace=True)

In [ ]:
y = df_nocovid["Earnings Per Share - Actual Surprise AbsVals"]
X = df_nocovid.loc[:, "Earnings Per Share – Coefficient of Variation":]
# perform the fixed effects panel regression, setting check_rank=False
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True, check_rank=False)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary
